In [19]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
print(f"Dataset downloaded to: {path}")


Dataset downloaded to: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2


In [21]:
import os

path = "/root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2"
print("Contents of the directory:")
print(os.listdir(path))



Contents of the directory:
['chest_xray']


In [25]:
import os

chest_xray_path = os.path.join(path, 'chest_xray')  # Combine the base path with the folder name
print("Contents of chest_xray:")
print(os.listdir(chest_xray_path))

train_path = os.path.join(chest_xray_path, 'train')  # Path to the training data
test_path = os.path.join(chest_xray_path, 'test')    # Path to the testing data
val_path = os.path.join(chest_xray_path, 'val')      # Path to the validation data (if exists)

print("Train path:", train_path)
print("Test path:", test_path)
print("Validation path:", val_path)




Contents of chest_xray:
['train', 'val', '__MACOSX', 'chest_xray', 'test']
Train path: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/train
Test path: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/test
Validation path: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/val


In [26]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [28]:
data_generators = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    shear_range = 0.2,
    validation_split = 0.2
)

train_data = data_generators.flow_from_directory(
    "/root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/train",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
    subset = 'training'
)

validation_data = data_generators.flow_from_directory(
    "/root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/val",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
    subset = 'validation'
)

testing_data = data_generators.flow_from_directory(
    "/root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/test",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 4173 images belonging to 2 classes.
Found 2 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [29]:

base_model = VGG16(weights = 'imagenet', include_top = False, input_shape=(224,224,3))

base_model.trainable = FALSE

model = Sequential(
    [base_model,
     Flatten(),
     Dense(128,activation='relu'),
     Dropout(0.2),
     Dense(1,activation='sigmoid')]
)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['Accuracy'])

In [31]:
model.fit(train_data, validation_data = validation_data, epochs = 4)

Epoch 1/4
131/131 ━━━━━━━━━━━━━━━━━━━━ 103s 753ms/step - Accuracy: 0.9026 - loss: 0.2603 - val_Accuracy: 1.0000 - val_loss: 0.0545
Epoch 2/4
131/131 ━━━━━━━━━━━━━━━━━━━━ 140s 737ms/step - Accuracy: 0.9033 - loss: 0.2445 - val_Accuracy: 0.5000 - val_loss: 0.3593
Epoch 3/4
131/131 ━━━━━━━━━━━━━━━━━━━━ 102s 749ms/step - Accuracy: 0.9147 - loss: 0.2212 - val_Accuracy: 0.5000 - val_loss: 2.6078
Epoch 4/4
131/131 ━━━━━━━━━━━━━━━━━━━━ 141s 742ms/step - Accuracy: 0.9178 - loss: 0.2022 - val_Accuracy: 1.0000 - val_loss: 0.0119


In [32]:
result = model.evaluate(testing_data)

print(f"Test Loss: {result[0]}, Test Accuracy: {result[1]}")

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 31s 968ms/step - Accuracy: 0.8558 - loss: 0.3494
Test Loss: 0.34925490617752075, Test Accuracy: 0.8557692170143127


In [39]:
model.save("pneumonia_prediction.h5")